# Data Preparation

In [1]:
import pandas as pd

In [6]:
# load data recognizing various NA values
df = pd.read_csv('../data/amazon_delivery.csv',  na_values=['NaN', 'NaN ', 'nan', 'nan ', 'null', ' null',  ''], skipinitialspace=True) # type: ignore

# strip leading/trailing spaces from all string columns
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
df.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


**Note**
After loading the dataset, extra spaces around string values and column names were removed, using .map() and .str.strip(). This ensured consistent formatting across the dataset for cleaner analysis.

## Step 1: Basic Data Inspection
To understand the structure, column types, and missing values. This helps guide the cleaning and feature engineering process.

In [8]:
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43739 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         43739 non-null  object 
 1   Agent_Age        43739 non-null  int64  
 2   Agent_Rating     43685 non-null  float64
 3   Store_Latitude   43739 non-null  float64
 4   Store_Longitude  43739 non-null  float64
 5   Drop_Latitude    43739 non-null  float64
 6   Drop_Longitude   43739 non-null  float64
 7   Order_Date       43739 non-null  object 
 8   Order_Time       43648 non-null  object 
 9   Pickup_Time      43739 non-null  object 
 10  Weather          43648 non-null  object 
 11  Traffic          43648 non-null  object 
 12  Vehicle          43739 non-null  object 
 13  Area             43739 non-null  object 
 14  Delivery_Time    43739 non-null  int64  
 15  Category         43739 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 5.3+ MB


Order_ID            0
Agent_Age           0
Agent_Rating       54
Store_Latitude      0
Store_Longitude     0
Drop_Latitude       0
Drop_Longitude      0
Order_Date          0
Order_Time         91
Pickup_Time         0
Weather            91
Traffic            91
Vehicle             0
Area                0
Delivery_Time       0
Category            0
dtype: int64

In [9]:
# get count of missing values >0 only
df.isnull().sum()[df.isnull().sum()>0]

Agent_Rating    54
Order_Time      91
Weather         91
Traffic         91
dtype: int64

Dataset has __43,739 rows__ and __16 columns__
Most columns have complete data, but __four have missing values__:
- Agent_Rating: 54 missing
- Order_Time: 91 missing
- Weather: 91 missing
- Traffic: 91 missing

Columns by Type:
- __Numerical (int/float)__: Agent_Age, Agent_Rating, Store_Latitude, Store_Longitude, Drop_Latitude, Drop_Longitude, Delivery_Time
- __Categorical (object)__: Order_ID, Order_Date, Order_Time, Pickup_Time, Weather, Traffic, Vehicle, Area, Category

## Step 2: Handling missing values

- **Agent_Rating (54 rows)**: Filled using the mean to preserve distribution
- **Order_Time (91 rows)**: Dropped to ensure clean datetime conversion
- **Weather (91 rows)**: Filled using the most frequent value (mode)
- **Traffic (91 rows)**: Filled using the most frequent value (mode)

These strategies balance data quality and retention for accurate feature engineering.

In [10]:
# fill missing agent ratings with the mean value
df['Agent_Rating'] = df['Agent_Rating'].fillna(df['Agent_Rating'].mean())

In [11]:
# drop missing order time (safe: only 91 of 43,739)
df = df.dropna(subset=['Order_Time'])

In [12]:
# fill missing weather values with the mode (most frequent value)
df['Weather'] = df['Weather'].fillna(df['Weather'].mode()[0])

In [13]:
# fill missing traffic values with the mode (most frequent value)
df['Traffic'] = df['Traffic'].fillna(df['Traffic'].mode()[0])

In [14]:
# get count of missing values >0 to confirm
df.isnull().sum()[df.isnull().sum()>0]

Series([], dtype: int64)

## Step 3: Datetime Conversion
- Combine Order_Date and Order_time into oder_datetime
- Convert Pickup_Time and Delivery_Time
- Create time duration features

This step is essential to enable time-based analysis of delivery performance, delays, and efficiency. It also allows us to engineer new features based on delivery timelines. 

In [40]:
# combine order_date and order_Time
df['order_datetime'] = pd.to_datetime(df['Order_Date'] + ' ' + df['Order_Time'])

# combine order_date and pickup_time first, and fix pickup_datetime if it is earlier than order_datetime (next day pickup)
df['pickup_datetime'] = pd.to_datetime(df['Order_Date'] + ' ' + df['Pickup_Time'])
df.loc[df['pickup_datetime'] < df['order_datetime'], 'pickup_datetime'] += pd.Timedelta(days=1)

# calculate delivery_datetime by adding Delivery_Time to pickup_datetime
df['delivery_datetime'] = df['pickup_datetime'] + pd.to_timedelta(df['Delivery_Time'], unit='m')

In [41]:
# preview the new datetime columns
df[['order_datetime', 'pickup_datetime', 'delivery_datetime']].head()

,order_datetime,pickup_datetime,delivery_datetime
0,2022-03-19 11:30:00,2022-03-19 11:45:00,2022-03-19 13:45:00
1,2022-03-25 19:45:00,2022-03-25 19:50:00,2022-03-25 22:35:00
2,2022-03-19 08:30:00,2022-03-19 08:45:00,2022-03-19 10:55:00
3,2022-04-05 18:00:00,2022-04-05 18:10:00,2022-04-05 19:55:00
4,2022-03-26 13:30:00,2022-03-26 13:45:00,2022-03-26 16:15:00


In [22]:
# check data types
df[['order_datetime', 'pickup_datetime', 'delivery_datetime']].dtypes

order_datetime       datetime64[ns]
pickup_datetime      datetime64[ns]
delivery_datetime    datetime64[ns]
dtype: object

**Note**

*pickup_datetime*
The initial datetime construction for `pickup_datetime` assumed that pickup always occurred on the same day as the order. However, some pickups occurred after midnight, resulting in a timestamp earlier than the order. To resolve this, a conditional adjustment was applied to shift `pickup_datetime` by one day when it fell before `order_datetime`.

*delivery_datetime*
Originally, Delivery_Time was assumed to represent minutes since the Unix epoch. However, after inspecting the column's values and previewing the output, it was confirmed that this field reflects that duration from pickup to delivery in minutes. To accurately reconstruct the delivery timestamp, Delivery_Time was added to pickup_datetime using pd.to_timedelta. This approach ensures proper temporal sequencing for duration-based analysis.

In [42]:
# time from order to pickup (in minutes)
df['order_to_pickup_mins'] = (df['pickup_datetime'] - df['order_datetime']).dt.total_seconds() / 60

# time from pickup to delivery (last mile)
df['pickup_to_delivery_mins'] = (df['delivery_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60

# total time from order to delivery
df['total_fulfillment_mins'] = (df['delivery_datetime'] - df['order_datetime']).dt.total_seconds() / 60

Additional features were created to capture key time intervals in the delivery process:

- `order_to_pickup_mins`: minutes between order placement and pickup
- `pickup_to_delivery_mins`: minutes between pickup and final delivery
- `total_fulfillment_mins`: total duration from order to delivery

## Step 4: Geospatial Feature Engineering

To support location-based analysis and evaluate route efficiency, a new feature 'distance_km' was engineered using the Haversine formula. This metric calculates the straight-line distance between the store and the drop-off location, based on their latitude and longitude coordinates.

To improve performance, the distance between store and drop-off locations was calculated using a vectorized implementation of the Haversine formula. This method leverages NumPy operations to compute distances across the dataset more efficiently than row-wise `.apply()`, while producing the same result.

The distance calculation using `.apply()` and the `haversine` package was compared with a NumPy vectorized version of the same formula. Minor differences in output were observed due to floating-point precision, but both methods produced functionally equivalent results for last-mile delivery analysis. The vectorized approach was preferred for performance scalability.


### Coordinate correction for Store_Latitude

Some rows had negative `Store_Latitude` values that placed the store in a different hemisphere than the drop-off location. A correction step was applied to flip the sign of `Store_Latitude` **only when doing so significantly reduced the calculated distance**.

In [52]:
# identify rows where Store_Latitude and Drop_Latitude have different signs
mask_diff_signs = (df['Store_Latitude'] * df['Drop_Latitude'] < 0)
df[mask_diff_signs==True]

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,...,Area,Delivery_Time,Category,order_datetime,pickup_datetime,delivery_datetime,distance_km,order_to_pickup_mins,pickup_to_delivery_mins,total_fulfillment_mins
90,hnme204254454,34,4.7,-27.163303,78.057044,27.233303,78.127044,2022-02-16,23:00:00,23:05:00,...,Metropolitian,75,Clothing,2022-02-16 23:00:00,2022-02-16 23:05:00,2022-02-17 00:20:00,6048.631240,5.0,75.0,80.0
275,mctx146223837,32,4.7,-27.165108,78.015053,27.225108,78.075053,2022-02-13,20:55:00,21:10:00,...,Metropolitian,155,Clothing,2022-02-13 20:55:00,2022-02-13 21:10:00,2022-02-13 23:45:00,6047.919478,15.0,155.0,170.0
1045,jxrf169692899,20,4.7,-15.546594,73.760431,15.606594,73.820431,2022-02-13,20:00:00,20:10:00,...,Metropolitian,145,Toys,2022-02-13 20:00:00,2022-02-13 20:10:00,2022-02-13 22:35:00,3464.082720,10.0,145.0,155.0
1705,csyn869746139,39,4.5,-23.230791,77.437020,23.250791,77.457020,2022-02-13,09:15:00,09:20:00,...,Metropolitian,145,Books,2022-02-13 09:15:00,2022-02-13 09:20:00,2022-02-13 11:45:00,5168.516553,5.0,145.0,150.0
1893,bocx711605816,25,4.0,-22.539129,88.365507,22.549129,88.375507,2022-02-15,09:25:00,09:30:00,...,Metropolitian,95,Toys,2022-02-15 09:25:00,2022-02-15 09:30:00,2022-02-15 11:05:00,5013.585658,5.0,95.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42071,lzux792298119,32,4.6,-15.498603,73.826911,15.568603,73.896911,2022-02-18,23:10:00,23:15:00,...,Metropolitian,120,Toys,2022-02-18 23:10:00,2022-02-18 23:15:00,2022-02-19 01:15:00,3454.524245,5.0,120.0,125.0
42252,lpkh473392828,22,5.0,-15.157944,73.950889,15.217944,74.010889,2022-02-17,18:50:00,18:55:00,...,Metropolitian,135,Sports,2022-02-17 18:50:00,2022-02-17 18:55:00,2022-02-17 21:10:00,3377.651072,5.0,135.0,140.0
42824,nnam562968751,23,4.8,-9.982834,76.283268,10.072834,76.373268,2022-02-14,19:20:00,19:35:00,...,Metropolitian,105,Sports,2022-02-14 19:20:00,2022-02-14 19:35:00,2022-02-14 21:20:00,2230.110757,15.0,105.0,120.0
43105,pvcj122440792,30,4.9,-19.874733,75.353942,19.904733,75.383942,2022-02-15,22:20:00,22:35:00,...,Metropolitian,75,Pet Supplies,2022-02-15 22:20:00,2022-02-15 22:35:00,2022-02-15 23:50:00,4423.276011,15.0,75.0,90.0


In [53]:
# flip Store_Latitude for those rows
df.loc[mask_diff_signs, 'Store_Latitude'] *= -1
df[mask_diff_signs==True]

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,...,Area,Delivery_Time,Category,order_datetime,pickup_datetime,delivery_datetime,distance_km,order_to_pickup_mins,pickup_to_delivery_mins,total_fulfillment_mins
90,hnme204254454,34,4.7,27.163303,78.057044,27.233303,78.127044,2022-02-16,23:00:00,23:05:00,...,Metropolitian,75,Clothing,2022-02-16 23:00:00,2022-02-16 23:05:00,2022-02-17 00:20:00,6048.631240,5.0,75.0,80.0
275,mctx146223837,32,4.7,27.165108,78.015053,27.225108,78.075053,2022-02-13,20:55:00,21:10:00,...,Metropolitian,155,Clothing,2022-02-13 20:55:00,2022-02-13 21:10:00,2022-02-13 23:45:00,6047.919478,15.0,155.0,170.0
1045,jxrf169692899,20,4.7,15.546594,73.760431,15.606594,73.820431,2022-02-13,20:00:00,20:10:00,...,Metropolitian,145,Toys,2022-02-13 20:00:00,2022-02-13 20:10:00,2022-02-13 22:35:00,3464.082720,10.0,145.0,155.0
1705,csyn869746139,39,4.5,23.230791,77.437020,23.250791,77.457020,2022-02-13,09:15:00,09:20:00,...,Metropolitian,145,Books,2022-02-13 09:15:00,2022-02-13 09:20:00,2022-02-13 11:45:00,5168.516553,5.0,145.0,150.0
1893,bocx711605816,25,4.0,22.539129,88.365507,22.549129,88.375507,2022-02-15,09:25:00,09:30:00,...,Metropolitian,95,Toys,2022-02-15 09:25:00,2022-02-15 09:30:00,2022-02-15 11:05:00,5013.585658,5.0,95.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42071,lzux792298119,32,4.6,15.498603,73.826911,15.568603,73.896911,2022-02-18,23:10:00,23:15:00,...,Metropolitian,120,Toys,2022-02-18 23:10:00,2022-02-18 23:15:00,2022-02-19 01:15:00,3454.524245,5.0,120.0,125.0
42252,lpkh473392828,22,5.0,15.157944,73.950889,15.217944,74.010889,2022-02-17,18:50:00,18:55:00,...,Metropolitian,135,Sports,2022-02-17 18:50:00,2022-02-17 18:55:00,2022-02-17 21:10:00,3377.651072,5.0,135.0,140.0
42824,nnam562968751,23,4.8,9.982834,76.283268,10.072834,76.373268,2022-02-14,19:20:00,19:35:00,...,Metropolitian,105,Sports,2022-02-14 19:20:00,2022-02-14 19:35:00,2022-02-14 21:20:00,2230.110757,15.0,105.0,120.0
43105,pvcj122440792,30,4.9,19.874733,75.353942,19.904733,75.383942,2022-02-15,22:20:00,22:35:00,...,Metropolitian,75,Pet Supplies,2022-02-15 22:20:00,2022-02-15 22:35:00,2022-02-15 23:50:00,4423.276011,15.0,75.0,90.0


In [57]:
import numpy as np

# convert degrees to radians
store_lat = np.radians(df['Store_Latitude'])
store_lon = np.radians(df['Store_Longitude'])
drop_lat = np.radians(df['Drop_Latitude'])
drop_lon = np.radians(df['Drop_Longitude'])

# Haversine formula (vectorized)
dlat = drop_lat - store_lat
dlon = drop_lon - store_lon

a = np.sin(dlat / 2)**2 + np.cos(store_lat) * np.cos(drop_lat) * np.sin(dlon / 2)**2
c = 2 * np.arcsin(np.sqrt(a))
earth_radius_km = 6371 # Earth's radius in km
df['distance_km'] = earth_radius_km * c

# preview the result
df[['Store_Latitude', 'Store_Longitude', 'Drop_Latitude', 'Drop_Longitude', 'distance_km']].head()


,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,distance_km
0,22.745049,75.892471,22.765049,75.912471,3.025149
1,12.913041,77.683237,13.043041,77.813237,20.183530
2,12.914264,77.678400,12.924264,77.688400,1.552758
3,11.003669,76.976494,11.053669,77.026494,7.790401
4,12.972793,80.249982,13.012793,80.289982,6.210138


**Note**

The `distance_km` feature was calculated using the Haversine formula, which represents straight-line distance. While this is useful for estimating proximity, it does not account for real-world routing constraints such as traffic, road infrastructure, or delivery logistics. 

## Step 5: Final Check & Export
- check column types
- check for any remaining missing values
- review new engineered features
- export the cleaned dataset as a csv

In [66]:
(df['order_to_pickup_mins'] < 0).sum(), (df['total_fulfillment_mins'] < 0).sum()

(np.int64(0), np.int64(0))

In [67]:
# confirm all column types are correct
df.dtypes

Order_ID                           object
Agent_Age                           int64
Agent_Rating                      float64
Store_Latitude                    float64
Store_Longitude                   float64
Drop_Latitude                     float64
Drop_Longitude                    float64
Order_Date                         object
Order_Time                         object
Pickup_Time                        object
Weather                            object
Traffic                            object
Vehicle                            object
Area                               object
Delivery_Time                       int64
Category                           object
order_datetime             datetime64[ns]
pickup_datetime            datetime64[ns]
delivery_datetime          datetime64[ns]
distance_km                       float64
order_to_pickup_mins              float64
pickup_to_delivery_mins           float64
total_fulfillment_mins            float64
dtype: object

In [68]:
# confirm no missing values remain
df.isnull().sum()[df.isnull().sum() > 0]

Series([], dtype: int64)

In [69]:
# confirm new features exist and look correct
df[['order_datetime', 'pickup_datetime', 'delivery_datetime', 
    'order_to_pickup_mins', 'pickup_to_delivery_mins', 'total_fulfillment_mins', 
    'distance_km']].describe()

,order_datetime,pickup_datetime,delivery_datetime,order_to_pickup_mins,pickup_to_delivery_mins,total_fulfillment_mins,distance_km
count,43648,43648,43648,43648.000000,43648.000000,43648.000000,43648.000000
mean,2022-03-14 09:55:13.388929536,2022-03-14 10:05:12.866568960,2022-03-14 12:10:07.735062016,9.991294,124.914475,134.905769,9.732144
min,2022-02-11 00:00:00,2022-02-11 00:05:00,2022-02-11 00:21:00,5.000000,10.000000,15.000000,1.465067
25%,2022-03-04 22:50:00,2022-03-04 23:00:00,2022-03-05 01:05:00,5.000000,90.000000,100.000000,4.663412
50%,2022-03-15 16:35:00,2022-03-15 16:45:00,2022-03-15 18:22:30,10.000000,125.000000,135.000000,9.220148
75%,2022-03-27 18:15:00,2022-03-27 18:26:15,2022-03-27 20:45:00,15.000000,160.000000,170.000000,13.681379
max,2022-04-06 23:55:00,2022-04-07 00:05:00,2022-04-07 03:40:00,15.000000,270.000000,285.000000,20.969489
std,NaN,NaN,NaN,4.086680,51.933163,52.055762,5.603598


In [70]:
# export the cleaned dataset
df.to_csv('../data/amazon_delivery_cleaned.csv', index=False)